# The Battle of the Neigbourhoods

In this notebook the neighbourhoods in the city of Toronto will be segmented en clustered based on the postalcode and borough information.<br><br>
Part 1: Collecting Toronto Neighbourhood data<br>
For the Toronto neighbourhood data, a Wikipedia page exists that has all the information that is neccessary to explore and cluster the neighbourhoods in Toronto.<br> The required HTML table from the Wikipedia page will be used to read into a pandas dataframe.<br><br>
Part 2: Collecting geographical coordinates of Toronto<br>
After cleaning en preprocessing of the data, it will be enriched with the geographical coordinates.<br><br>
Part 3: Clustering the neighbourhoods of Toronto<br>
Then the venues in each neighbourhood will be collected form the FOURSQUARE website. The top 10 venues of each neighbourhoods will be determined.<br> 
The K-means algoritm is used to cluster the neighbourhoods, which are then visualized in a map of Toronto using Folium.<br>

## Part 1: Collecting the Toronto neighbourhood data.

### Importing libraries

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from bs4 import BeautifulSoup # Library for scraping webpage
from IPython.display import display_html # Library for displaying HTML

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Importing ready!')

Importing ready!


### Retreive neighbourhoods of Amsterdam from Wikipedia webpage

In [4]:
# Get webpage
source = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Amsterdam').text
# Scrape webpage
soup = BeautifulSoup(source,'lxml')
# Check title of webpage
print(soup.title)
lst = []
for item in soup.findAll('div',{'class':'mw-category-group'}):
    sub_items = item.findAll('li')
    for sub_item in sub_items:
        lst.append(['Amsterdam',sub_item.text])
        
# Get table from webpage
#html_table = str(bsoup.table)
# Display table
#display_html(html_table,raw=True)
lst

<title>Category:Neighbourhoods of Amsterdam - Wikipedia</title>


[['Amsterdam', 'Template:Neighborhoods of Amsterdam'],
 ['Amsterdam', 'Admiralenbuurt'],
 ['Amsterdam', 'Amsteldorp'],
 ['Amsterdam', 'Amsterdam Oud-West'],
 ['Amsterdam', 'Amsterdam Oud-Zuid'],
 ['Amsterdam', 'Amsterdam Science Park'],
 ['Amsterdam', 'Apollobuurt'],
 ['Amsterdam', 'Betondorp'],
 ['Amsterdam', 'Bijlmermeer'],
 ['Amsterdam', 'Binnenstad (Amsterdam)'],
 ['Amsterdam', 'Bos en Lommer'],
 ['Amsterdam', 'Buiksloot'],
 ['Amsterdam', 'Buikslotermeer'],
 ['Amsterdam', 'Buitenveldert'],
 ['Amsterdam', 'Bullewijk'],
 ['Amsterdam', 'Burgwallen Nieuwe Zijde'],
 ['Amsterdam', 'Burgwallen Oude Zijde'],
 ['Amsterdam', 'Chassébuurt'],
 ['Amsterdam', 'Cruquiuseiland'],
 ['Amsterdam', 'Czaar Peterbuurt'],
 ['Amsterdam', 'Dapperbuurt'],
 ['Amsterdam', 'De Aker'],
 ['Amsterdam', 'De Pijp'],
 ['Amsterdam', 'De Wallen'],
 ['Amsterdam', 'Diamantbuurt (Amsterdam)'],
 ['Amsterdam', 'Duivelseiland (Amsterdam)'],
 ['Amsterdam', 'Eastern Docklands'],
 ['Amsterdam', 'Eendracht (Amsterdam)'],
 ['Ams

In [5]:
df=pd.DataFrame(lst,columns=['City','Neighbourhood'])
df["Neighbourhood"]  = df["Neighbourhood"].str.strip()
df

,City,Neighbourhood
0,Amsterdam,Template:Neighborhoods of Amsterdam
1,Amsterdam,Admiralenbuurt
2,Amsterdam,Amsteldorp
3,Amsterdam,Amsterdam Oud-West
4,Amsterdam,Amsterdam Oud-Zuid
...,...,...
102,Amsterdam,Westerpark (neighbourhood)
103,Amsterdam,Willemspark (Amsterdam)
104,Amsterdam,Zeeburgereiland
105,Amsterdam,Zeeheldenbuurt


In [51]:
df.shape

(107, 2)

### Cleaning en preparing dataset

In [52]:
df.loc[df.Neighbourhood.str.contains('Template'), 'Neighbourhood'].count() # df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'].count()

1

### Convert HTML table to Dataframe for preprocessing

In [66]:
df1 = df[~df.Neighbourhood.str.contains('Template')]
df1

,City,Neighbourhood
1,Amsterdam,Admiralenbuurt
2,Amsterdam,Amsteldorp
3,Amsterdam,Amsterdam Oud-West
4,Amsterdam,Amsterdam Oud-Zuid
5,Amsterdam,Amsterdam Science Park
...,...,...
102,Amsterdam,Westerpark (neighbourhood)
103,Amsterdam,Willemspark (Amsterdam)
104,Amsterdam,Zeeburgereiland
105,Amsterdam,Zeeheldenbuurt


In [67]:
df1.shape

(106, 2)

In [70]:
df1.loc[df1.Neighbourhood.str.contains('\('), 'Neighbourhood']

Series([], Name: Neighbourhood, dtype: object)

In [69]:
df1['Neighbourhood'] = df1['Neighbourhood'].str.split('\(').str[0]
#df1['Neighbourhood'] = df1['Neighbourhood'].str.replace('\(Amsterdam\)','')
#df1['Neighbourhood'] = df1['Neighbourhood'].str.replace('\(neighbourhood\)','')
df1

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,City,Neighbourhood
1,Amsterdam,Admiralenbuurt
2,Amsterdam,Amsteldorp
3,Amsterdam,Amsterdam Oud-West
4,Amsterdam,Amsterdam Oud-Zuid
5,Amsterdam,Amsterdam Science Park
...,...,...
102,Amsterdam,Westerpark
103,Amsterdam,Willemspark
104,Amsterdam,Zeeburgereiland
105,Amsterdam,Zeeheldenbuurt


In [35]:
df1.loc[df1.Neighbourhood.str.contains('\('), 'Neighbourhood']

9               Binnenstad (Amsterdam)
24            Diamantbuurt (Amsterdam)
25           Duivelseiland (Amsterdam)
27               Eendracht (Amsterdam)
34         Haarlemmerbuurt (Amsterdam)
51               Molenwijk (Amsterdam)
52          Museumkwartier (Amsterdam)
63         Oosterparkbuurt (Amsterdam)
77               Rapenburg (Amsterdam)
79           Rivierenbuurt (Amsterdam)
84         Slotervaart (neighbourhood)
86       Staatsliedenbuurt (Amsterdam)
89                Teleport (Amsterdam)
90          Transvaalbuurt (Amsterdam)
94               Uilenburg (Amsterdam)
98            Weesperzijde (Amsterdam)
100    Westelijke Eilanden (Amsterdam)
102         Westerpark (neighbourhood)
103            Willemspark (Amsterdam)
Name: Neighbourhood, dtype: object

In [ ]:
df_list = pd.read_html(html_table)
df = df_list[0]
df.rename(columns={'Postal Code':'Postcode'},inplace=True)
print(df.shape)
df